In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from src import *
from dataset.data_utils import get_CIFAR10_data

In [2]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in list(data.items()):
  print(('%s: ' % k, v.shape))

('X_val: ', (1000, 3, 32, 32))
('y_val: ', (1000,))
('y_test: ', (1000,))
('X_test: ', (1000, 3, 32, 32))
('y_train: ', (49000,))
('X_train: ', (49000, 3, 32, 32))


In [9]:
layers = [
    Affine_layer(100),
    Batch_norm_layer(),
    Relu_layer(),
]
model = Net(layers, 3 * 32 * 32, 10, reg = 0.01)
optim = [Sgd(1e-3), Sgd(1e-3)]
solver = Solver(model, data, verbose = True, batch_size = 200, print_every = 100, lr_decay = 0.95,
                optim = optim)
solver.train()
solver.check_accuracy(data['X_test'], data['y_test'])

(Iteration 1 / 2450) loss: 69.379361
(Epoch 0 / 10) train acc: 0.103000; val_acc: 0.098000
(Iteration 101 / 2450) loss: 25015.644379
(Iteration 201 / 2450) loss: 50806.071292
(Epoch 1 / 10) train acc: 0.090000; val_acc: 0.112000


KeyboardInterrupt: 